<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Выгрузка-данных" data-toc-modified-id="Выгрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Выгрузка данных</a></span></li><li><span><a href="#Результаты-исследования" data-toc-modified-id="Результаты-исследования-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Результаты исследования</a></span></li></ul></div>

# Анализ NPS для телекоммуникационной компании

## Выгрузка данных

Загрузим необходимые библиотеки

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Сформируем обращение к БД
path_to_db = 'telecomm_csi.db' # путь к файлу
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
# SQL запрос

query = """
SELECT u.user_id AS user_id,
        u.lt_day AS lt_day,
        u.age AS age,
        CASE
			WHEN u.gender_segment = 1 THEN 'men'
			WHEN u.gender_segment = 0 THEN 'women'
		END AS gender_segment,
        u.cpe_type_name AS cpe_type_name,
        u.nps_score AS nps_score,
		u.os_name AS os_name,
		CASE
			WHEN u.lt_day <= 365 THEN 'new'
			WHEN u.lt_day > 365 THEN 'old'
		END AS is_new,
		CASE
			WHEN u.nps_score >=9 THEN 'promoters'
			WHEN u.nps_score >=7 THEN 'passives'
			ELSE 'detractors'
		END AS nps_group,
		l.country,
		l.city,
		a.title AS age_gr_id,
		lt.title AS lifetime_segment,
		t.title AS traffic_segment
FROM user AS u
LEFT JOIN location AS l ON u.location_id = l.location_id
LEFT JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id
LEFT JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
WHERE u.gender_segment IS NOT NULL
"""

In [4]:
# Выгружаем данные в датасет df
df = pd.read_sql(query, engine)
df.head()

,user_id,lt_day,age,gender_segment,cpe_type_name,nps_score,os_name,is_new,nps_group,country,city,age_gr_id,lifetime_segment,traffic_segment
0,A001A2,2320,45.0,men,SMARTPHONE,10,ANDROID,old,promoters,Россия,Уфа,05 45-54,08 36+,04 1-5
1,A001WF,2344,53.0,women,SMARTPHONE,10,ANDROID,old,promoters,Россия,Киров,05 45-54,08 36+,04 1-5
2,A003Q7,467,57.0,women,SMARTPHONE,10,ANDROID,old,promoters,Россия,Москва,06 55-64,06 13-24,08 20-25
3,A004TB,4190,44.0,men,SMARTPHONE,10,IOS,old,promoters,Россия,РостовнаДону,04 35-44,08 36+,03 0.1-1
4,A004XT,1163,24.0,women,SMARTPHONE,10,ANDROID,old,promoters,Россия,Рязань,02 16-24,08 36+,05 5-10


In [5]:
# Сохраняем таблицу локально для Tableau
df.to_csv('dash_nps_telecom.csv', sep='\t')

## Результаты исследования

**Дашборд**


[Структура данных опроса](https://public.tableau.com/app/profile/sofya4724/viz/Datastructure_16572198908430/datastructure?publish=yes)

[Результаты изучения NPS](https://public.tableau.com/app/profile/sofya4724/viz/NPSdynamics/dynamicsNPS?publish=yes)

**Краткий отчет по результатам работы**

https://disk.yandex.ru/i/fvTZrF0sHQMMKg